# info_strada_sports_scrape

This python script pulls athlete profile information from certain `infostradasports.com` pages. It obviously will work on the currently set `INFOSTRADASPORTS_URL` url and likely on other pages with a matching template / design.

## Global Variables

- `INFOSTRADASPORTS_URL` : URL which data is pulled from.
- `NUM_OF_ATHLETE_PROFILES_TO_CAPTURE` : number of profiles to capture. Set this to `0` to capture all profiles from page

## Imports

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from pprint import pprint

## Global Varialbes

In [2]:
NUM_OF_ATHLETE_PROFILES_TO_CAPTURE = 20
INFOSTRADASPORTS_URL = 'http://ipc.infostradasports.com/asp/lib/TheASP.asp?pageid=8903&sportid=514&NOCClubID=-1&Olympic=0&WinterGames=-1&ContinentGeoID=-1'

## functions

In [3]:
def get_unique_keys(athlete_profiles):
    athlete_profile_keys = set()
    for athlete_profile in athlete_profiles:
        for key in athlete_profile:
            athlete_profile_keys.add(key)
    return athlete_profile_keys

def initialize_database(unique_keys):
    athlete_database = {}
    for key in unique_keys:
        athlete_database[key] = []
    return athlete_database
        
def fill_database(database, athlete_profiles):
    for athlete_profile in athlete_profiles:
        for key_d, value_d in database.items():
            if key_d in athlete_profile:
                database[key_d].append(athlete_profile[key_d])
            else:
                database[key_d].append('--')
    return database

            
def get_athlete_database(athlete_profiles):
    unique_keys = get_unique_keys(athlete_profiles)
    database = initialize_database(unique_keys)
    database = fill_database(database, athlete_profiles)
    return database

def fixup_whitespace(text):
    return " ".join(text.split())


# --- scrape functions --- #
    
def scrape_athlete_profile(bio_table):
    bio_table_trs = bio_table.find_all('tr')
    details = {}
    for index, tr in enumerate(bio_table_trs):
        if index == 0:
            details['Name'] = fixup_whitespace(tr.get_text())
        else:
            tds = tr.find_all('td')
            details[tds[0].get_text()] = fixup_whitespace(tds[1].get_text())
    return details

def scrape_human_interest_info(bio_table):
    bio_table_trs = bio_table.find_all('tr')
    details = {}
    for index, tr in enumerate(bio_table_trs):
        if len(tr.find_all('td')) == 1:
            continue
        else:
            tds = tr.find_all('td')
            details[tds[0].get_text()] = fixup_whitespace(tds[1].get_text())
    return details


def scrape_competition_highlights_info(bio_table):
    pass

## Initialize web driver, visit `INFOSTRADASPORTS_URL` and capture links

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/home/ff/.wdm/drivers/chromedriver/linux64/103.0.5060.134/chromedriver] found in cache
/tmp/ipykernel_66721/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [5]:
driver.get(INFOSTRADASPORTS_URL)
links = driver.find_elements_by_xpath("//a[@href]")

/tmp/ipykernel_66721/731036308.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  links = driver.find_elements_by_xpath("//a[@href]")


## Script

In [6]:
athlete_profiles = []
human_interest_infos = []
competition_highlights_infos = []

for index, link in enumerate(links):
    if index == 0:
        continue
    else:
        link.click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        bio_tables = soup.find_all('table', {'class':'table_layout1'})
        athlete_profile_soup = bio_tables[0]
        human_interest_info_soup = bio_tables[1]
        
        athlete_profile = scrape_athlete_profile(athlete_profile_soup)
        athlete_profiles.append(athlete_profile)
        
        human_interest_info = scrape_human_interest_info(human_interest_info_soup)
        human_interest_infos.append(human_interest_info)
        
        driver.back()
    if NUM_OF_ATHLETE_PROFILES_TO_CAPTURE == 0:
        continue
    elif index == NUM_OF_ATHLETE_PROFILES_TO_CAPTURE:
        break


athlete_database = get_athlete_database(athlete_profiles)
df1 = pd.DataFrame(athlete_database)


athlete_interest_database = get_athlete_database(human_interest_infos)
df2 = pd.DataFrame(athlete_interest_database)


df_final = pd.concat([df1, df2], axis=1)
df_final.to_csv('infostradasports_scrape_data.csv')

In [7]:
df_final

,Place of Birth,NPC,Name,Age,Height,Sport,Gender,Why this sport?,Type of Impairment,Tournament,...,Higher education,Competing for,Awards and honours,Other information,Nicknames,Residence,Family,Training Regime,Origin of Impairment,When and where did you begin this sport?
0,--,Afghanistan,HAIDARI Zubair,43,--,"Para athletics, Para swimming, Shooting Para s...",Men,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1,--,Afghanistan,PARWANI Khan Agha,37,--,Para swimming,Men,--,--,--,...,--,--,--,--,--,--,--,--,--,--
2,--,Andorra,CODINA MOLINE Marc,51,--,Para swimming,Men,--,--,--,...,--,--,--,--,--,--,--,--,--,--
3,--,Andorra,SANCHEZ FRANCISCO Antonio,37,--,Para swimming,Men,--,--,--,...,--,--,--,NATIONAL FIRSTHe became the first person to re...,--,--,--,--,--,--
4,--,Angola,LOPES Silvio Mendes,29,--,Para swimming,Men,--,Limb deficiency,--,...,"Universidade Tecnica de Angola: Luanda, ANG",--,--,--,--,"Luanda, ANG",--,He trains at the First of August Swimming Pool...,--,--
5,--,Angola,MORAIS Jandira Q. Paixao,34,--,Para swimming,Women,--,--,--,...,--,--,--,STUDY FIRSTShe did not compete in competitions...,--,"Luanda, ANG",--,--,--,--
6,"Río Gallegos, ARG",Argentina,ALONSO Milagros,21,--,Para swimming,Women,"""I tried swimming, and found that I felt more ...",Intellectual impairment,--,...,--,--,--,--,"Mili (elmediadortv.com.ar, 13 Mar 2017)",--,--,--,Congenital,She began swimming at age six.
7,"Villa Carlos Paz, ARG",Argentina,ARAGÓN Jazmín,21,--,Para swimming,Women,Her doctor suggested the sport to her after sh...,Impaired range of motion,--,...,Fashion Design - University of Buenos Aires [U...,--,--,--,"Jaz (Facebook profile, 12 Apr 2019)","Villa Carlos Paz, ARG",--,--,Congenital,She began swimming at age seven in Villa Carlo...
8,"Lobería, ARG",Argentina,ARAYA Elián,31,1.77 m,Para swimming,Men,He chose the sport following a medical recomme...,Intellectual impairment,--,...,--,Argentina,--,--,--,--,--,--,--,"He began swimming in 2002 in Loberia, Argentina."
9,--,Argentina,ARCE Facundo Matias,25,--,Para swimming,Men,--,Cerebral Palsy,--,...,--,--,In 2013 he was named Best Para-Athlete in the ...,--,--,"Trelew, ARG",--,--,--,--
